# TPS-Aug-2022

In [1]:
class Config:
    NB = '109'
    dataset_NB = '108'

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'id'
    target = 'failure'

## Import libralies

In [2]:
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)


color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import KNNImputer

## Load data

In [5]:
df_train = pd.read_pickle(f"../data/processed/nb{Config.dataset_NB}_train.pkl", compression='zip')
df_test = pd.read_pickle(f"../data/processed/nb{Config.dataset_NB}_test.pkl", compression='zip')

df_dataset = pd.concat(objs=[df_train, df_test], axis=0).reset_index(drop=True)
df_dataset

,id,product_code,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,missing_loading,missing_measurement_3,missing_measurement_4,missing_measurement_5,missing_measurement_9,ohe0_5,ohe0_7,ohe1_5,ohe1_6,ohe1_7,ohe1_8
0,0,A,80.10,9,5,7.0,8.0,11.0,18.040,12.518000,15.748,19.292000,11.739,20.155,10.672,15.859000,17.594,15.193,15.029,14.537333,13.034,14.684000,764.100,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
1,1,A,84.89,9,5,14.0,3.0,11.0,18.213,11.540000,17.717,17.893000,12.748,17.889,12.448,17.947000,17.915,11.755,14.732,15.425000,14.395,15.631000,682.057,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
2,2,A,82.43,9,5,12.0,1.0,11.0,18.057,11.652000,16.738,18.240000,12.718,18.288,12.715,15.607000,19.391,13.798,16.711,18.631000,14.094,17.946000,663.376,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
3,3,A,101.07,9,5,13.0,2.0,11.0,17.295,11.188000,18.576,18.339000,12.583,19.060,12.471,16.346000,18.377,10.020,15.250,15.562000,16.154,17.172000,826.282,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
4,4,A,188.06,9,5,9.0,2.0,11.0,19.346,12.950000,16.990,15.746000,11.306,18.093,10.337,17.082000,19.932,12.428,16.182,12.760000,13.153,16.412000,579.885,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,47340,I,144.74,9,5,0.0,4.0,11.0,18.465,12.570000,18.146,17.089000,11.204,18.573,11.691,15.664333,19.771,11.562,17.246,15.131000,15.209,16.027667,696.466,NaN,False,False,False,False,False,0.0,1.0,1.0,0.0,0.0,0.0
47341,47341,I,74.53,9,5,4.0,8.0,11.0,18.900,9.896000,18.288,18.713333,11.747,17.917,10.980,16.027000,15.694,13.564,15.494,15.296000,13.812,16.501000,613.249,NaN,False,False,False,False,False,0.0,1.0,1.0,0.0,0.0,0.0
47342,47342,I,67.73,9,5,10.0,11.0,11.0,18.656,12.439667,18.242,17.910000,12.081,19.630,10.436,16.137000,20.612,11.134,16.519,15.525000,14.175,17.728000,783.349,NaN,False,False,True,False,False,0.0,1.0,1.0,0.0,0.0,0.0
47343,47343,I,126.15,9,5,8.0,16.0,11.0,16.536,11.226000,18.144,17.250000,12.692,19.575,12.672,15.422000,19.496,9.319,15.817,17.403000,16.437,15.179000,745.210,NaN,False,False,False,False,False,0.0,1.0,1.0,0.0,0.0,0.0


## 各列の積と差（絶対値）をとる

In [6]:
for i in range(18):
    for j in range(18):
        if i >= j:
            continue

        df_dataset[f'mes_{i}x{j}'] = df_dataset[f'measurement_{i}'] * df_dataset[f'measurement_{j}']
        df_dataset[f'mes_{i}-{j}'] = (df_dataset[f'measurement_{i}'] - df_dataset[f'measurement_{j}']).abs()

In [7]:
df_dataset

,id,product_code,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,missing_loading,missing_measurement_3,missing_measurement_4,missing_measurement_5,missing_measurement_9,ohe0_5,ohe0_7,ohe1_5,ohe1_6,ohe1_7,ohe1_8,mes_0x1,mes_0-1,mes_0x2,mes_0-2,mes_0x3,mes_0-3,mes_0x4,mes_0-4,mes_0x5,mes_0-5,mes_0x6,mes_0-6,mes_0x7,mes_0-7,mes_0x8,mes_0-8,mes_0x9,mes_0-9,mes_0x10,mes_0-10,mes_0x11,mes_0-11,mes_0x12,mes_0-12,mes_0x13,mes_0-13,mes_0x14,mes_0-14,mes_0x15,mes_0-15,mes_0x16,mes_0-16,mes_0x17,mes_0-17,mes_1x2,mes_1-2,mes_1x3,mes_1-3,mes_1x4,mes_1-4,mes_1x5,mes_1-5,mes_1x6,mes_1-6,mes_1x7,mes_1-7,mes_1x8,mes_1-8,mes_1x9,mes_1-9,mes_1x10,mes_1-10,mes_1x11,mes_1-11,mes_1x12,mes_1-12,mes_1x13,mes_1-13,mes_1x14,mes_1-14,mes_1x15,mes_1-15,mes_1x16,mes_1-16,mes_1x17,mes_1-17,mes_2x3,mes_2-3,mes_2x4,mes_2-4,mes_2x5,mes_2-5,mes_2x6,mes_2-6,mes_2x7,mes_2-7,mes_2x8,mes_2-8,mes_2x9,mes_2-9,mes_2x10,mes_2-10,mes_2x11,mes_2-11,mes_2x12,mes_2-12,mes_2x13,mes_2-13,mes_2x14,mes_2-14,mes_2x15,mes_2-15,mes_2x16,mes_2-16,mes_2x17,mes_2-17,mes_3x4,mes_3-4,mes_3x5,mes_3-5,mes_3x6,mes_3-6,mes_3x7,mes_3-7,mes_3x8,mes_3-8,mes_3x9,mes_3-9,mes_3x10,mes_3-10,mes_3x11,mes_3-11,mes_3x12,mes_3-12,mes_3x13,mes_3-13,mes_3x14,mes_3-14,mes_3x15,mes_3-15,mes_3x16,mes_3-16,mes_3x17,mes_3-17,mes_4x5,mes_4-5,mes_4x6,mes_4-6,mes_4x7,mes_4-7,mes_4x8,mes_4-8,mes_4x9,mes_4-9,mes_4x10,mes_4-10,mes_4x11,mes_4-11,mes_4x12,mes_4-12,mes_4x13,mes_4-13,mes_4x14,mes_4-14,mes_4x15,mes_4-15,mes_4x16,mes_4-16,mes_4x17,mes_4-17,mes_5x6,mes_5-6,mes_5x7,mes_5-7,mes_5x8,mes_5-8,mes_5x9,mes_5-9,mes_5x10,mes_5-10,mes_5x11,mes_5-11,mes_5x12,mes_5-12,mes_5x13,mes_5-13,mes_5x14,mes_5-14,mes_5x15,mes_5-15,mes_5x16,mes_5-16,mes_5x17,mes_5-17,mes_6x7,mes_6-7,mes_6x8,mes_6-8,mes_6x9,mes_6-9,mes_6x10,mes_6-10,mes_6x11,mes_6-11,mes_6x12,mes_6-12,mes_6x13,mes_6-13,mes_6x14,mes_6-14,mes_6x15,mes_6-15,mes_6x16,mes_6-16,mes_6x17,mes_6-17,mes_7x8,mes_7-8,mes_7x9,mes_7-9,mes_7x10,mes_7-10,mes_7x11,mes_7-11,mes_7x12,mes_7-12,mes_7x13,mes_7-13,mes_7x14,mes_7-14,mes_7x15,mes_7-15,mes_7x16,mes_7-16,mes_7x17,mes_7-17,mes_8x9,mes_8-9,mes_8x10,mes_8-10,mes_8x11,mes_8-11,mes_8x12,mes_8-12,mes_8x13,mes_8-13,mes_8x14,mes_8-14,mes_8x15,mes_8-15,mes_8x16,mes_8-16,mes_8x17,mes_8-17,mes_9x10,mes_9-10,mes_9x11,mes_9-11,mes_9x12,mes_9-12,mes_9x13,mes_9-13,mes_9x14,mes_9-14,mes_9x15,mes_9-15,mes_9x16,mes_9-16,mes_9x17,mes_9-17,mes_10x11,mes_10-11,mes_10x12,mes_10-12,mes_10x13,mes_10-13,mes_10x14,mes_10-14,mes_10x15,mes_10-15,mes_10x16,mes_10-16,mes_10x17,mes_10-17,mes_11x12,mes_11-12,mes_11x13,mes_11-13,mes_11x14,mes_11-14,mes_11x15,mes_11-15,mes_11x16,mes_11-16,mes_11x17,mes_11-17,mes_12x13,mes_12-13,mes_12x14,mes_12-14,mes_12x15,mes_12-15,mes_12x16,mes_12-16,mes_12x17,mes_12-17,mes_13x14,mes_13-14,mes_13x15,mes_13-15,mes_13x16,mes_13-16,mes_13x17,mes_13-17,mes_14x15,mes_14-15,mes_14x16,mes_14-16,mes_14x17,mes_14-17,mes_15x16,mes_15-16,mes_15x17,mes_15-17,mes_16x17,mes_16-17
0,0,A,80.10,9,5,7.0,8.0,11.0,18.040,12.518000,15.748,19.292000,11.739,20.155,10.672,15.859000,17.594,15.193,15.029,14.537333,13.034,14.684000,764.100,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0,56.0,1.0,77.0,4.0,126.280,11.040,87.626000,5.518000,110.236,8.748,135.044000,12.292000,82.173,4.739,141.085,13.155,74.704,3.672,111.013,8.859000,123.158,10.594,106.351,8.193,105.203,8.029,101.761333,7.537333,91.238,6.034,102.788,7.684000,5348.700,757.100,88.0,3.0,144.320,10.040,100.144000,4.518000,125.984,7.748,154.336000,11.292000,93.912,3.739,161.240,12.155,85.376,2.672,126.872000,7.859000,140.752,9.594,121.544,7.193,120.232,7.029,116.298667,6.537333,104.272,5.034,117.472000,6.684000,6112.800,756.100,198.440,7.040,137.698000,1.518000,173.228,4.748,212.212000,8.292000,129.129,0.739,221.705,9.155,117.3

## Save Dataset

In [8]:
## Separate train dataset and test dataset
train_len = len(df_train)

train = df_dataset[:train_len]
test = df_dataset[train_len:]
test.drop(columns=[Config.target], inplace=True)

In [9]:
Config.NB

'109'

In [10]:
pd.to_pickle(train, Config.processed_data_dir + f"nb{Config.NB}_train.pkl", compression='zip')
pd.to_pickle(test, Config.processed_data_dir + f"nb{Config.NB}_test.pkl", compression='zip')

## 検証メモ